In [1]:
!pip install transformers[torch]
!pip install accelerate -U
! pip install transformers datasets
!pip install -q datasets transformers evaluate

from huggingface_hub import notebook_login

notebook_login()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
#To utilize the HuggingFace dataset, you might consider employing the following cell.
'''
hf_dataset_identifier = "segments/sidewalk-semantic"
from datasets import load_dataset

ds = load_dataset(hf_dataset_identifier)

ds = ds.shuffle(seed=1)
ds = ds["train"].train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]
'''

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

'''
import json
from huggingface_hub import cached_download, hf_hub_url

repo_id = "huggingface/label-files"
filename = "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

'''

In [3]:

import h5py
import zipfile
import os
import numpy as np
from datasets import Dataset, DatasetDict
from PIL import Image, ImageEnhance

# Specify your zip file's path
zip_path = '/content/drive/MyDrive/Colab Notebooks/brainTumorDataPublic_2299-3064.zip'

# Create a ZipFile object
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents of the zip file in current directory
    zip_ref.extractall()

# Get the list of .mat files
mat_files = [file for file in os.listdir() if file.endswith(".mat")]

data_store = {'pixel_values': [], 'label': [], 'tumor_category': []}

# Load all .mat files
for file in mat_files:
    with h5py.File(file, 'r') as f:
        # 'cjdata' is the key in the .mat file
        cjdata = f['cjdata']
        # Access the 'image', 'label', and 'tumorMask' datasets within 'cjdata'
        image = np.array(cjdata['image'])
        label = np.array(cjdata['label'])
        tumorMask = np.array(cjdata['tumorMask'])

        # Convert the NumPy arrays back into PIL image objects
        image = Image.fromarray((image / np.max(image) * 255).astype('uint16'))
        tumorMask = Image.fromarray((tumorMask / np.max(tumorMask) * 1).astype('uint16'))

        # Convert the grayscale image to a three-channel color image
        image = image.convert('RGB')
        tumorMask = tumorMask.convert('L')

        # Add the 'image', 'label', and 'tumorMask' data to the data store
        data_store['pixel_values'].append(image)
        data_store['label'].append(tumorMask)
        data_store['tumor_category'].append(label)

# Convert the data store to a Hugging Face Dataset
ds = Dataset.from_dict(data_store)

# Remove the 'scene_category' column
ds = ds.remove_columns('tumor_category')

# Split the data into training and testing sets
ds = ds.train_test_split(test_size=0.2)
#ds=ds[1:40]
# Now you have your training and testing datasets
train_ds = ds["train"]
test_ds = ds["test"]
val_ds=test_ds
# Convert the Dataset to a dictionary
#train_ds = { 'pixel_values': [img for img in train_ds['image']], 'label': [mask for mask in train_ds['annotation']] }
#test_ds = { 'pixel_values': [img for img in test_ds['image']], 'label': [mask for mask in test_ds['annotation']] }

# Get the unique labels from 'scene_category'



In [31]:

from tensorflow.keras.optimizers import Adam

from transformers import (
    DefaultDataCollator,
    SegformerFeatureExtractor,
    TFSegformerForSemanticSegmentation,
    create_optimizer,
)
from transformers.keras_callbacks import KerasMetricCallback


In [32]:

feature_extractor = SegformerFeatureExtractor()


def transforms(image):
    image = tf.keras.utils.img_to_array(image)
    image = image.transpose((2, 0, 1))
    return image


def preprocess(example_batch):
    images = [transforms(x.convert("RGB")) for x in example_batch["pixel_values"]]
    labels = [x for x in example_batch["label"]]
    inputs = feature_extractor(images, labels)
    return inputs



In [33]:
# Get the first image in the 'label' field
first_image = val_ds['label'][0]

# Convert the image to a NumPy array and print the pixel values
print(np.array(first_image))


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:

#Display the pixel condition of the mask and image using the cell
# Convert the image to a NumPy array
'''
image_array = np.array(first_image)

# Get the unique pixel values and their counts
unique_pixels, counts = np.unique(image_array, return_counts=True)

# Print the unique pixel values and their counts
for pixel, count in zip(unique_pixels, counts):
    print(f"Pixel value: {pixel}, Count: {count}")

'''


Pixel value: 0, Count: 258307
Pixel value: 150, Count: 3837


In [ ]:
#ignore this block
'''
from datasets import Features, Image

def change_label(example):
    # Change label value for tumor from 255 to 150
    example['label'] = np.where(np.array(example['label']) == 255, 150, example['label'])
    return example

# Define the features of your dataset
features = Features({
    'pixel_values': Image(shape=(None, None, 3), dtype='uint8'),
    'label': Image(shape=(None, None, 1), dtype='uint8')
})

# Apply the function to the 'label' field of your datasets
train_ds = train_ds.map(change_label, features=features)
test_ds = test_ds.map(change_label, features=features)
val_ds = val_ds.map(change_label, features=features)
''''

In [34]:

train_ds.set_transform(preprocess)
val_ds.set_transform(preprocess)


In [35]:

import json

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datasets import load_dataset, load_metric
from huggingface_hub import cached_download, hf_hub_url
from tensorflow.keras.optimizers import Adam


In [48]:

epochs = 5
lr = 0.000006
batch_size = 16


In [49]:
data_collator = DefaultDataCollator(return_tensors="tf")

train_set = train_ds.to_tf_dataset(
    columns=["pixel_values", "label"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

val_set = val_ds.to_tf_dataset(
    columns=["pixel_values", "label"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)



In [37]:

# Look at the shapes of each features within the dataset.
train_set.element_spec



{'labels': TensorSpec(shape=(None, 512, 512), dtype=tf.int64, name=None),
 'pixel_values': TensorSpec(shape=(None, 3, 512, 512), dtype=tf.float32, name=None)}

In [38]:

label2id = {"background": 0, "tumor": 150}
id2label = {0: "background", 150: "tumor"}


In [39]:
num_labels=len(id2label)
num_labels

2

In [50]:

# Investigate a single batch.
batch = next(iter(train_set))
batch["pixel_values"].shape, batch["labels"].shape



(TensorShape([16, 3, 512, 512]), TensorShape([16, 512, 512]))

In [41]:

batch["labels"]



<tf.Tensor: shape=(32, 512, 512), dtype=int64, numpy=
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 

In [51]:

metric = load_metric("mean_iou")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # logits are of shape (batch_size, num_labels, height, width), so
    # we first transpose them to (batch_size, height, width, num_labels)
    logits = tf.transpose(logits, perm=[0, 2, 3, 1])
    # scale the logits to the size of the label
    logits_resized = tf.image.resize(
        logits,
        size=tf.shape(labels)[1:],
        method="bilinear",
    )
    # compute the prediction labels and compute the metric
    pred_labels = tf.argmax(logits_resized, axis=-1)
    metrics = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=num_labels,
        ignore_index=-1,
        #reduce_labels=feature_extractor.reduce_labels,
    )
    return {"val_" + k: v for k, v in metrics.items()}



metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset=val_set,
    batch_size=batch_size,
    label_cols=["labels"],
)




/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [19]:
#Please ignore this cell
'''
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = tf.transpose(logits, perm=[0, 2, 3, 1])
    logits_resized = tf.image.resize(
        logits,
        size=tf.shape(labels)[1:],
        method="bilinear",
    )
    pred_labels = tf.argmax(logits_resized, axis=-1)
    # Change labels to 'tumor_1', 'tumor_2', etc.
    labels = tf.strings.as_string(labels, prefix='tumor_')
    metrics = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=num_labels,
        ignore_index=-1,
        reduce_labels=feature_extractor.reduce_labels,
    )
    return {"val_" + k: v for k, v in metrics.items()}
'''

In [52]:

model_checkpoint = "nvidia/mit-b0"
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=False,  # Will ensure the segmentation specific components are reinitialized.
)

optimizer = Adam(learning_rate=lr)
model.compile(optimizer)



Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.summary()


Model: "tf_segformer_for_semantic_segmentation_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 segformer (TFSegformerMain  multiple                  3319392   
 Layer)                                                          
                                                                 
 decode_head (TFSegformerDe  multiple                  395778    
 codeHead)                                                       
                                                                 
Total params: 3715170 (14.17 MB)
Trainable params: 3714658 (14.17 MB)
Non-trainable params: 512 (2.00 KB)
_________________________________________________________________


In [54]:


# Dummy forward pass.
outputs = model(batch)
outputs.loss, outputs.logits.shape


(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.61049366], dtype=float32)>,
 TensorShape([16, 2, 128, 128]))

In [55]:

callbacks = [metric_callback]


In [56]:

# To keep the runtime of this example short, we will train it for only two epochs.
# For real applications purposes, note that you will likely need a longer training schedule.
model.fit(
    train_set,
    validation_data=val_set,
    callbacks=callbacks,
    epochs=epochs,
)

Epoch 1/5
39/39 [==============================] - 423s 10s/step - loss: 0.5887 - val_loss: 0.3855 - val_mean_iou: 0.4835 - val_mean_accuracy: 0.6544 - val_overall_accuracy: 0.8949 - val_per_category_iou: 0.4835 - val_per_category_accuracy: 0.6544
Epoch 2/5
39/39 [==============================] - 375s 10s/step - loss: 0.4501 - val_loss: 0.3155 - val_mean_iou: 0.5206 - val_mean_accuracy: 0.6722 - val_overall_accuracy: 0.9328 - val_per_category_iou: 0.5206 - val_per_category_accuracy: 0.6722
Epoch 3/5
39/39 [==============================] - 360s 9s/step - loss: 0.3511 - val_loss: 0.2611 - val_mean_iou: 0.5560 - val_mean_accuracy: 0.6717 - val_overall_accuracy: 0.9581 - val_per_category_iou: 0.5560 - val_per_category_accuracy: 0.6717
Epoch 4/5
39/39 [==============================] - 356s 9s/step - loss: 0.2898 - val_loss: 0.1984 - val_mean_iou: 0.5722 - val_mean_accuracy: 0.6148 - val_overall_accuracy: 0.9759 - val_per_category_iou: 0.5722 - val_per_category_accuracy: 0.6148
Epoch 5/5
